In [2]:
from preprocess_functions import *

import torch
#import pytorch_lightning as pl

# PARAMETRY
current_dir = os.getcwd()#os.path.dirname(os.path.realpath(__file__))
INPUT_FILE = os.path.join(current_dir, "sentiment.txt")
OUTPUT_DIR = current_dir

In [5]:
X_text[0], Y[0]

('I hate Harry Potter, that daniel wotshisface needs a fucking slap...', 0)

In [3]:
X_text , Y = get_data(INPUT_FILE)
token_idx,num_tokens = tokenize_text(X_text)
max_tokens = get_max(X_text)

input_sequences = create_sequences(X_text, token_idx,max_tokens)
input_sequences, type(input_sequences)

(array([[  0,   0,   0, ...,  47,  85, 299],
        [  0,   0,   0, ..., 150,  58, 304],
        [  0,   0,   0, ...,  58, 360,  95],
        ...,
        [  0,   0,   0, ..., 293, 212, 170],
        [  0,   0,   0, ..., 212, 335, 389],
        [  0,   0,   0, ..., 212, 335, 389]]),
 numpy.ndarray)

In [6]:
input_sequences = torch.from_numpy(input_sequences)
input_sequences, type(input_sequences)

(tensor([[  0,   0,   0,  ...,  47,  85, 299],
         [  0,   0,   0,  ..., 150,  58, 304],
         [  0,   0,   0,  ...,  58, 360,  95],
         ...,
         [  0,   0,   0,  ..., 293, 212, 170],
         [  0,   0,   0,  ..., 212, 335, 389],
         [  0,   0,   0,  ..., 212, 335, 389]], dtype=torch.int32),
 torch.Tensor)

In [7]:
input_sequences.shape

torch.Size([7086, 40])

In [48]:
y_num = np.array(Y)
y_ten = torch.tensor(y_num)
y_ten.shape

torch.Size([7086])

In [8]:
input_sequences[0]

tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 440, 368, 331, 308, 232, 407, 363, 205,  47,  85, 299],
       dtype=torch.int32)

In [38]:
import torch.nn as nn
import torch.optim as optim


class FeedForward(nn.Module):
    def __init__(self, num_tokens, max_tokens):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_tokens, embedding_dim=8, max_norm=max_tokens)
        self.gru1 = nn.GRU(input_size=8, hidden_size=16, batch_first = True)
        self.gru2 = nn.GRU(input_size=16, hidden_size=8, batch_first = True)
        self.gru3 = nn.GRU(input_size=8, hidden_size=4, batch_first = True)
        self.linear = nn.Linear(in_features=4, out_features=1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x, _ = self.gru3(x)
        x = self.linear(x[:, -1, :])
        x = self.sigmoid(x)
        return x

In [39]:
model = FeedForward(num_tokens, max_tokens)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

In [60]:

batch_size = 2

input_data = torch.randint(0, num_tokens, (batch_size, max_tokens))
labels = torch.randint(0, 2, (batch_size, 1)).float()

In [61]:
input_data.shape, labels.shape

(torch.Size([2, 40]), torch.Size([2, 1]))

In [62]:
labels

tensor([[0.],
        [0.]])

In [68]:
input_data=input_sequences
labels = y_ten.unsqueeze(1)

In [69]:
input_data.shape, labels.shape

(torch.Size([7086, 40]), torch.Size([7086, 1]))

In [71]:
def train_model(model, optimizer, criterion, input_data, labels, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(input_data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [72]:
num_epochs = 100
train_model(model, optimizer, criterion, input_data, labels, num_epochs)

RuntimeError: Found dtype Int but expected Float